In [4]:
import pandas as pd
import numpy as np
import json
from chinese_calendar import is_workday, is_holiday

In [5]:
base = '../../data/'

In [6]:
test_query = pd.read_csv(base + 'processed/test_od.csv')
test_plans = pd.read_pickle(base + 'processed/test_plan_change_format.pkl')

In [7]:
test_query.head()

,sid,pid,req_time,o,d,o_lon,o_lat,d_lon,d_lat,o_i,o_j,d_i,d_j
0,1126541,178395.0,2018-12-05 14:51:43,"116.41,39.92","116.46,39.95",116.41,39.92,116.46,39.95,23.0,11.0,25.0,12.0
1,453685,179178.0,2018-12-01 12:50:33,"116.39,39.90","116.41,39.91",116.39,39.90,116.41,39.91,23.0,10.0,24.0,11.0
2,2066073,184748.0,2018-12-07 10:17:35,"116.37,39.71","116.31,39.89",116.37,39.71,116.31,39.89,22.0,6.0,21.0,10.0
3,452746,162795.0,2018-12-01 07:52:44,"116.33,39.97","116.31,39.98",116.33,39.97,116.31,39.98,21.0,12.0,21.0,12.0
4,1431672,116384.0,2018-12-06 10:36:30,"116.51,39.68","116.51,39.68",116.51,39.68,116.51,39.68,26.0,5.0,26.0,5.0


In [9]:
test_plans.head()

,sid,agg_dis,agg_price,agg_eta,agg_mode,mode_num,first_dis,first_price,first_eta,first_mode,last_mode,mean_dis,min_dis,max_dis,mean_price,min_price,max_price,mean_eta,min_eta,max_eta
0,420123,"[22691, 21483, 21483, 21328]","[700.0, -1.0, 5500.0, 400.0]","[4888, 2303, 2543, 6046]","[7, 3, 4, 1]",4,22691,700.0,4888,7,1,21746.25,21328,22691,1649.75,-1.0,5500.0,3945.0,2303,6046
1,420125,"[20310, 20310, 21465, 21541, 20419]","[-1.0, 5900.0, 500.0, 700.0, 400.0]","[1484, 1784, 3925, 4397, 5206]","[3, 4, 9, 7, 1]",5,20310,-1.0,1484,3,1,20809.00,20310,21541,1499.80,-1.0,5900.0,3359.2,1484,5206
2,420127,"[22706, 21745, 21745, 21372]","[700.0, -1.0, 5700.0, 400.0]","[4883, 2233, 2413, 6267]","[7, 3, 4, 1]",4,22706,700.0,4883,7,1,21892.00,21372,22706,1699.75,-1.0,5700.0,3949.0,2233,6267
3,420129,"[22378, 20658, 20658, 21555, 20964]","[700.0, -1.0, 5300.0, 500.0, 400.0]","[4594, 2165, 2465, 3861, 5693]","[7, 3, 4, 9, 1]",5,22378,700.0,4594,7,1,21242.60,20658,22378,1379.80,-1.0,5300.0,3755.6,2165,5693
4,420134,"[1148, 1148, 1046, 1046]","[-1.0, 1300.0, -1.0, -1.0]","[125, 125, 315, 895]","[3, 4, 6, 5]",4,1148,-1.0,125,3,5,1097.00,1046,1148,324.25,-1.0,1300.0,365.0,125,895


In [10]:
test_query['pid'].fillna(-1, inplace=True)
test_query['pid_exist'] = 1
test_query['pid_exist'][(test_query.pid==-1)] = 0

test_query['req_time'] = pd.to_datetime(test_query['req_time'])
test_query['hod'] = test_query['req_time'].dt.hour
test_query['dow'] = test_query['req_time'].dt.dayofweek + 1
test_query.head()

/home/caizekun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,sid,pid,req_time,o,d,o_lon,o_lat,d_lon,d_lat,o_i,o_j,d_i,d_j,pid_exist,hod,dow
0,1126541,178395.0,2018-12-05 14:51:43,"116.41,39.92","116.46,39.95",116.41,39.92,116.46,39.95,23.0,11.0,25.0,12.0,1,14,3
1,453685,179178.0,2018-12-01 12:50:33,"116.39,39.90","116.41,39.91",116.39,39.90,116.41,39.91,23.0,10.0,24.0,11.0,1,12,6
2,2066073,184748.0,2018-12-07 10:17:35,"116.37,39.71","116.31,39.89",116.37,39.71,116.31,39.89,22.0,6.0,21.0,10.0,1,10,5
3,452746,162795.0,2018-12-01 07:52:44,"116.33,39.97","116.31,39.98",116.33,39.97,116.31,39.98,21.0,12.0,21.0,12.0,1,7,6
4,1431672,116384.0,2018-12-06 10:36:30,"116.51,39.68","116.51,39.68",116.51,39.68,116.51,39.68,26.0,5.0,26.0,5.0,1,10,4


In [11]:
test_query['holiday'] = test_query.req_time.apply(lambda x:is_holiday(x)*1)
test_query['holiday'][(test_query.dow==6) | (test_query.dow==7)] = 1
test_query.head()

/home/caizekun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,sid,pid,req_time,o,d,o_lon,o_lat,d_lon,d_lat,o_i,o_j,d_i,d_j,pid_exist,hod,dow,holiday
0,1126541,178395.0,2018-12-05 14:51:43,"116.41,39.92","116.46,39.95",116.41,39.92,116.46,39.95,23.0,11.0,25.0,12.0,1,14,3,0
1,453685,179178.0,2018-12-01 12:50:33,"116.39,39.90","116.41,39.91",116.39,39.90,116.41,39.91,23.0,10.0,24.0,11.0,1,12,6,1
2,2066073,184748.0,2018-12-07 10:17:35,"116.37,39.71","116.31,39.89",116.37,39.71,116.31,39.89,22.0,6.0,21.0,10.0,1,10,5,0
3,452746,162795.0,2018-12-01 07:52:44,"116.33,39.97","116.31,39.98",116.33,39.97,116.31,39.98,21.0,12.0,21.0,12.0,1,7,6,1
4,1431672,116384.0,2018-12-06 10:36:30,"116.51,39.68","116.51,39.68",116.51,39.68,116.51,39.68,26.0,5.0,26.0,5.0,1,10,4,0


In [12]:
test_query.sort_values('req_time', inplace=True)
test_merge = pd.merge(test_query, test_plans, how='outer')
test_merge.fillna(-1, inplace=True)
test_merge.head()

,sid,pid,req_time,o,d,o_lon,o_lat,d_lon,d_lat,o_i,...,last_mode,mean_dis,min_dis,max_dis,mean_price,min_price,max_price,mean_eta,min_eta,max_eta
0,447255,-1.0,2018-12-01 00:00:10,"116.61,39.92","116.68,39.93",116.61,39.92,116.68,39.93,28.0,...,7.0,9355.800000,7140.0,11769.0,799.600000,-1.0,2900.0,1977.400000,1200.0,2867.0
1,468412,126418.0,2018-12-01 00:00:25,"116.37,39.87","116.37,39.86",116.37,39.87,116.37,39.86,22.0,...,6.0,145.000000,144.0,147.0,-1.000000,-1.0,-1.0,59.333333,14.0,122.0
2,421621,109311.0,2018-12-01 00:00:30,"116.31,39.94","116.27,39.91",116.31,39.94,116.27,39.91,21.0,...,1.0,9790.000000,7495.0,11448.0,716.333333,-1.0,3300.0,2289.833333,1031.0,3540.0
3,446843,-1.0,2018-12-01 00:00:35,"116.42,39.95","116.40,39.95",116.42,39.95,116.40,39.95,23.0,...,5.0,2702.666667,1918.0,3832.0,316.166667,-1.0,1500.0,1118.166667,450.0,1741.0
4,468884,-1.0,2018-12-01 00:00:40,"116.55,39.76","116.45,39.97",116.55,39.76,116.45,39.97,26.0,...,1.0,35168.200000,32081.0,37634.0,2859.800000,-1.0,10800.0,4482.200000,2537.0,6561.0


In [13]:
test_merge.loc[0]

sid                                        447255
pid                                            -1
req_time                      2018-12-01 00:00:10
o                                    116.61,39.92
d                                    116.68,39.93
o_lon                                      116.61
o_lat                                       39.92
d_lon                                      116.68
d_lat                                       39.93
o_i                                            28
o_j                                            11
d_i                                            31
d_j                                            11
pid_exist                                       0
hod                                             0
dow                                             6
holiday                                         1
agg_dis           [9044, 9044, 7140, 9782, 11769]
agg_price      [-1.0, 2900.0, -1.0, 300.0, 800.0]
agg_eta            [1200, 1200, 2158, 2462, 2867]


In [14]:
test_merge.to_pickle('../../data/processed/test_merge_od.pkl')